بسم الله والحمدلله والصلاة والسلام على رسول الله





In [8]:
import pandas as pd
from sklearn.metrics import confusion_matrix
import seaborn as sns
import matplotlib.pyplot as plt

In [9]:
files = {
    "Camel-CA": "/content/predictions_ANERtestC10_Camel-ca-ner.txt",
    "Camel-MSA": "/content/predictions_ANERtestC10_Camel-msa-ner.txt",
    "Camel-Mix": "/content/predictions_ANERtestC10_Camel-mix-ner.txt",
    "Multilingual": "/content/predictions_ANERtestC10_Multilingual2.txt",
    "Ensemble-Majority": "/content/Compined_Predictions_Majority_ANER.txt",
    "Ensemble-Weighted": "/content/Compined_Predictions_Weighted_ANER.txt"
}


In [11]:
import pandas as pd


def load_predictions(file_path):
    df = pd.read_csv(file_path, sep="\t", usecols=["Token","Gold Tag","Predicted Tag"])
    return df.dropna()

# جمع كل البيانات
all_tags = set()
tag_counts = {}
accuracies = {model: {} for model in files.keys()}

# حساب Count لكل tag
for model, file in files.items():
    df = load_predictions(file)
    for tag in df["Gold Tag"].unique():
        all_tags.add(tag)
        if tag not in tag_counts:
            tag_counts[tag] = (df["Gold Tag"] == tag).sum()
    # حساب الدقة لكل tag
    for tag in all_tags:
        correct = df[(df["Gold Tag"]==tag) & (df["Gold Tag"]==df["Predicted Tag"])].shape[0]
        total = df["Gold Tag"].value_counts().get(tag,0)
        accuracies[model][tag] = correct/total if total>0 else 0

# بناء الجدول النهائي
rows = []
for tag in sorted(all_tags):
    row = [tag, tag_counts[tag]]
    for model in files.keys():
        row.append(round(accuracies[model][tag], 2))
    rows.append(row)

columns = ["Tag", "Count"] + list(files.keys())
final_df = pd.DataFrame(rows, columns=columns)

# عرض الجدول داخل Colab
#display(final_df)

# عرض الجدول بصيغة Markdown
print(final_df.to_markdown(index=False))


| Tag    |   Count |   Camel-CA |   Camel-MSA |   Camel-Mix |   Multilingual |   Ensemble-Majority |   Ensemble-Weighted |
|:-------|--------:|-----------:|------------:|------------:|---------------:|--------------------:|--------------------:|
| B-LOC  |     380 |       0.96 |        0.97 |        0.92 |           0.99 |                0.97 |                0.98 |
| B-MISC |     126 |       0.76 |        0.91 |        0.91 |           0.83 |                0.91 |                0.91 |
| B-ORG  |     206 |       0.87 |        0.99 |        0.98 |           0.99 |                0.99 |                0.99 |
| B-PERS |     367 |       0.96 |        0.98 |        0.98 |           0.98 |                0.98 |                0.98 |
| I-LOC  |      25 |       0.88 |        0.92 |        0.96 |           0.96 |                0.96 |                0.96 |
| I-MISC |      44 |       0.75 |        0.93 |        0.89 |           0.75 |                0.93 |                0.93 |
| I-ORG  |     1